<a href="https://colab.research.google.com/github/ariarusso/AppliedTimeSeriesAnalysisWithPython/blob/main/Copia_di_bertGiulia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import transformers
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import pandas as pd
import gzip
import json
import os
import shutil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizerFast
import transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_name = "bert-base-uncased"
max_length = 128

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [ ]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):  #prendo il dato per intero e creo un dataframe
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

folder_path = os.path.join('.', 'reviews_Home_and_Kitchen_5.json.gz') 
dataset=getDF(folder_path)

In [ ]:
dataset.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,APYOBQE6M18AA,0615391206,Martin Schwartz,"[0, 0]",My daughter wanted this book and the price on ...,5.0,Best Price,1382140800,"10 19, 2013"
1,A1JVQTAGHYOL7F,0615391206,Michelle Dinh,"[0, 0]",I bought this zoku quick pop for my daughterr ...,5.0,zoku,1403049600,"06 18, 2014"
2,A3UPYGJKZ0XTU4,0615391206,mirasreviews,"[26, 27]",There is no shortage of pop recipes available ...,4.0,"Excels at Sweet Dessert Pops, but Falls Short ...",1367712000,"05 5, 2013"
3,A2MHCTX43MIMDZ,0615391206,"M. Johnson ""Tea Lover""","[14, 18]",This book is a must have if you get a Zoku (wh...,5.0,Creative Combos,1312416000,"08 4, 2011"
4,AHAI85T5C2DH3,0615391206,PugLover,"[0, 0]",This cookbook is great. I have really enjoyed...,4.0,A must own if you own the Zoku maker...,1402099200,"06 7, 2014"


In [ ]:
from transformers import BertTokenizerFast, BertForSequenceClassification

In [ ]:
df=dataset.copy()

In [ ]:
df['reviews']=df['reviewText']+df['summary']

In [ ]:
def to_sentiments(row): 
    if row['overall'] >= 0.0 and row['overall'] <= 3.0:
        val= 'Negativo'
    else:
        val='Positivo'
    return val

In [ ]:
df['sentiment'] = df.apply(to_sentiments, axis=1)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,reviews,sentiment
0,APYOBQE6M18AA,0615391206,Martin Schwartz,"[0, 0]",My daughter wanted this book and the price on ...,5.0,Best Price,1382140800,"10 19, 2013",My daughter wanted this book and the price on ...,Positivo
1,A1JVQTAGHYOL7F,0615391206,Michelle Dinh,"[0, 0]",I bought this zoku quick pop for my daughterr ...,5.0,zoku,1403049600,"06 18, 2014",I bought this zoku quick pop for my daughterr ...,Positivo
2,A3UPYGJKZ0XTU4,0615391206,mirasreviews,"[26, 27]",There is no shortage of pop recipes available ...,4.0,"Excels at Sweet Dessert Pops, but Falls Short ...",1367712000,"05 5, 2013",There is no shortage of pop recipes available ...,Positivo
3,A2MHCTX43MIMDZ,0615391206,"M. Johnson ""Tea Lover""","[14, 18]",This book is a must have if you get a Zoku (wh...,5.0,Creative Combos,1312416000,"08 4, 2011",This book is a must have if you get a Zoku (wh...,Positivo
4,AHAI85T5C2DH3,0615391206,PugLover,"[0, 0]",This cookbook is great. I have really enjoyed...,4.0,A must own if you own the Zoku maker...,1402099200,"06 7, 2014",This cookbook is great. I have really enjoyed...,Positivo


In [ ]:
df = df[['reviews','sentiment']]

In [ ]:
df.head()

,reviews,sentiment
0,My daughter wanted this book and the price on ...,Positivo
1,I bought this zoku quick pop for my daughterr ...,Positivo
2,There is no shortage of pop recipes available ...,Positivo
3,This book is a must have if you get a Zoku (wh...,Positivo
4,This cookbook is great. I have really enjoyed...,Positivo


In [ ]:
#Trasformiamo le label da categoriche in numeriche:
df['sentiment']= LabelEncoder().fit_transform(df['sentiment']) 
df['sentiment'].unique() 

array([1, 0])

## Dataset PRONTO ALL'USO

In [ ]:
dataset=df

In [ ]:
target_names=[0,1]
news_text = dataset.reviews
labels = dataset.sentiment
#(train_texts,valid_texts,train_labels,valid_labels)=train_test_split(news_text, labels, test_size=0.3)

In [ ]:
train, test = train_test_split(dataset,  test_size=0.2)

In [ ]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [ ]:
train.head()

,index,reviews,sentiment
0,387501,It's a TARDIS mug! Fantastical and true to the...,1
1,240768,We've had this for about 1 1/2 years. My wife ...,0
2,133502,"I have had three of these, maybe four. The pla...",0
3,117045,I've owned this microwave for 5 years and have...,1
4,501115,"I heard of this toaster on Cooks Country, the ...",1


In [ ]:
texts=[]

for row in train.reviews: 
    texts.append(row)

In [ ]:
train_encodings = tokenizer(texts, truncation=True,  max_length=max_length)

In [ ]:
#from transformers import  TFBertForSequenceClassification

In [ ]:
target_names=[0,1]

In [ ]:
model=BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    num_train_epochs=10,
    per_device_train_batch_size=8,  # batch size per device during training
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=False,
    logging_steps=200,
    evaluation_strategy="steps",
    output_dir='pretrain_bert'
)

In [ ]:
train= train.drop(['index'], axis = 1) 

In [ ]:
train

,reviews,sentiment
0,It's a TARDIS mug! Fantastical and true to the...,1
1,We've had this for about 1 1/2 years. My wife ...,0
2,"I have had three of these, maybe four. The pla...",0
3,I've owned this microwave for 5 years and have...,1
4,"I heard of this toaster on Cooks Country, the ...",1
...,...,...
441340,My husband works outside in the summer heat an...,1
441341,one arrived damaged and we did not do anything...,1
441342,I like these stones the best mainly because th...,1
441343,We ate at a wonderful restaurant in Italy and ...,1


In [ ]:
trainer = Trainer( model=model, args=training_args, train_dataset=train, eval_dataset=test )


In [ ]:
print(train.get('250799'))

None


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 441345
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 551690


KeyError: ignored

In [ ]:
trainer.save_model()

In [ ]:
# Keep track of train and evaluate loss.
loss_history = {'train_loss':[], 'eval_loss':[]}

# Keep track of train and evaluate perplexity.
# This is a metric useful to track for language models.
perplexity_history = {'train_perplexity':[], 'eval_perplexity':[]}

# Loop through each log history.
for log_history in trainer.state.log_history:

  if 'loss' in log_history.keys():
    # Deal with trianing loss.
    loss_history['train_loss'].append(log_history['loss'])
    perplexity_history['train_perplexity'].append(math.exp(log_history['loss']))
    
  elif 'eval_loss' in log_history.keys():
    # Deal with eval loss.
    loss_history['eval_loss'].append(log_history['eval_loss'])
    perplexity_history['eval_perplexity'].append(math.exp(log_history['eval_loss']))

# Plot Losses.
plot_dict(loss_history, start_step=training_args.logging_steps, 
          step_size=training_args.logging_steps, use_title='Loss', 
          use_xlabel='Train Steps', use_ylabel='Values', magnify=2)

print()

# Plot Perplexities.
plot_dict(perplexity_history, start_step=training_args.logging_steps, 
          step_size=training_args.logging_steps, use_title='Perplexity', 
          use_xlabel='Train Steps', use_ylabel='Values', magnify=2)

## **Evaluate**

For the final evaluation we can have a separate test set that we use to do our final perplexity evaluation. For simplicity I used the same validation text file for the final evaluation. That is the reason I get the same results as the last validation perplexity plot value.

In [ ]:
# check if `do_eval` flag is set.
if training_args.do_eval:
  
  # capture output if trainer evaluate.
  eval_output = trainer.evaluate()
  # compute perplexity from model loss.
  perplexity = math.exp(eval_output["eval_loss"])
  print('\nEvaluate Perplexity: {:10,.2f}'.format(perplexity))
else:
  print('No evaluation needed. No evaluation data provided, `do_eval=False`!')